In [1]:
# In[1]: Cell 1 - 필요한 모듈 및 경로 설정
import sys
import os

# WorkFrame 폴더의 경로를 추가하여 스텝별 스크립트를 import 할 수 있도록 함
# 이 노트북 파일이 'root/notebooks/'에 있다고 가정
workframe_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'WorkFrame'))
if workframe_path not in sys.path:
    sys.path.append(workframe_path)

import run_step5_prepare_gnn_data as step5
import run_step6_train_gnn_model as step6

E:\pyspace\envs\polygnn\Lib\site-packages\torch_geometric\typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: [WinError 127] 지정된 프로시저를 찾을 수 없습니다
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
E:\pyspace\envs\polygnn\Lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] 지정된 프로시저를 찾을 수 없습니다
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
E:\pyspace\envs\polygnn\Lib\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [WinError 127] 지정된 프로시저를 찾을 수 없습니다
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
E:\pyspace\envs\polygnn\Lib\site-packages\torch_geometric\typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: [WinError 127

In [3]:
# In[2]: Cell 2 - 5단계 실행: GNN 데이터셋 준비
# imputed_final_data.csv 파일을 읽어 GNN이 사용할 수 있는 그래프 데이터셋으로 변환하고 저장합니다.
# 이 셀은 한 번만 실행하면 됩니다.
step5.main()

--- 5단계 시작: GNN 데이터셋 준비 ---
대상 물성: ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
SMILES를 그래프 데이터로 변환 중... (시간이 걸릴 수 있습니다)
그래프 변환 완료. 총 10343개의 유효한 분자 데이터 생성.

--- 5단계 완료 ---
처리된 데이터셋이 'E:/SSL-polyGNN/data/processed/gnn_dataset.pt'에 저장되었습니다.


In [4]:
# In[3]: Cell 3 - 6단계 실행: GNN 모델 학습
# 위에서 생성한 데이터셋을 불러와 K-fold 교차 검증으로 모델을 학습합니다.
# 학습 과정(Epoch별 Loss)이 이 셀의 출력 창에 표시됩니다.
# 학습이 완료되면 최종 성능과 함께 모델 파일이 'root/model/' 폴더에 저장됩니다.
step6.main()

--- 6단계 시작: GNN 모델 학습 ---
사용 장치: cuda

===== Fold 1/5 =====
Epoch 001 | Train Loss: 2430.7221 | Val Loss: 1217.2559
Epoch 010 | Train Loss: 972.9030 | Val Loss: 786.5198
Epoch 020 | Train Loss: 906.1099 | Val Loss: 714.5302
Epoch 030 | Train Loss: 866.0552 | Val Loss: 685.7071
Epoch 040 | Train Loss: 812.4205 | Val Loss: 656.8478
Epoch 050 | Train Loss: 798.6108 | Val Loss: 652.9719
Epoch 060 | Train Loss: 776.2268 | Val Loss: 690.1607
Epoch 070 | Train Loss: 760.3277 | Val Loss: 609.2947
Epoch 080 | Train Loss: 741.0899 | Val Loss: 594.0068
Epoch 090 | Train Loss: 750.9163 | Val Loss: 606.0130
Epoch 100 | Train Loss: 727.1239 | Val Loss: 574.3458
Fold 1 최적 Val Loss: 573.4292

===== Fold 2/5 =====
Epoch 001 | Train Loss: 2429.3624 | Val Loss: 1321.4793
Epoch 010 | Train Loss: 1020.8406 | Val Loss: 866.0327
Epoch 020 | Train Loss: 947.6723 | Val Loss: 679.1697
Epoch 030 | Train Loss: 872.3544 | Val Loss: 676.3289
Epoch 040 | Train Loss: 842.8516 | Val Loss: 687.0255
Epoch 050 | Train Lo

In [ ]:
# In[4]: Cell 4 - (선택 사항) 학습된 모델로 예측해보기
import torch
import pandas as pd

# myutility 폴더 경로 추가
myutility_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'myutility'))
if myutility_path not in sys.path:
    sys.path.append(myutility_path)

import gnn_utils as gu

# --- 설정 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 5가지 물성 예측
NUM_TARGETS = 5
# 1번 Fold에서 가장 성능이 좋았던 모델을 불러옵니다.
MODEL_PATH = '../model/best_model_fold_1.pth'
# 테스트해볼 SMILES
TEST_SMILES = "CCO" # Ethanol

# --- 모델 로드 ---
model = gu.GCNNet(in_channels=36, hidden_channels=128, out_channels=NUM_TARGETS)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# --- 예측 ---
# 예측을 위해서는 y값이 필요 없으므로 임의의 값을 넣어줌
graph_data = gu.smiles_to_graph(TEST_SMILES, torch.zeros(NUM_TARGETS)).to(device)

with torch.no_grad():
    prediction = model(graph_data)

print(f"SMILES '{TEST_SMILES}'의 예측 물성값:")
# 원본 데이터의 물성 순서대로 출력 (예시)
properties = ['Density', 'FFV', 'Rg', 'Tc', 'Tg']
for name, val in zip(properties, prediction.squeeze().cpu().numpy()):
    print(f"  - {name}: {val:.4f}")